In [2]:
%run utils.py

In [3]:
from utils import ConvRelu
from torch import nn

def vgg_block(conv_in, conv_out, conv_num):
    layers = []
    layers.append(ConvRelu(conv_in, conv_out, kernel_size=3, padding=1))
    for i in range(conv_num - 1):
        layers.append(ConvRelu(conv_out, conv_out, kernel_size=3, padding=1))

    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

block = vgg_block(3, 64, 2)
print(block)

Sequential(
  (0): ConvRelu(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (1): ConvRelu(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [11]:
class VggBlock(nn.Module):
    def __init__(self, conv_in, conv_out, conv_num):
        super(VggBlock, self).__init__()
        self.layers = []
        self.layers.append(ConvRelu(conv_in, conv_out, kernel_size=3, padding=1))
        for i in range(conv_num - 1):
            self.layers.append(ConvRelu(conv_out, conv_out, kernel_size=3, padding=1))

        self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

block = VggBlock(3, 64, 2)
print(block)

VggBlock()


In [22]:
from utils import LinerRelu

class Vgg16(nn.Module):
    def __init__(self, dropout=0.5):
        super(Vgg16, self).__init__()
        self.block1 = VggBlock(3, 64, 2)
        self.block2 = VggBlock(64, 128, 2)
        self.block3 = VggBlock(128, 256, 3)
        self.block4 = VggBlock(256, 512, 3)
        self.block5 = VggBlock(512, 512, 3)
        self.fc1 = LinerRelu(512, 256, dropout=dropout)
        self.fc2 = LinerRelu(256, 128, dropout=dropout)
        self.fc3 = LinerRelu(128, 10, dropout=dropout)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [23]:
from torchinfo import summary

vgg16 = Vgg16()
summary(vgg16, (1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
Vgg16                                    [1, 10]                   --
├─VggBlock: 1-1                          [1, 64, 16, 16]           --
├─VggBlock: 1-2                          [1, 128, 8, 8]            --
├─VggBlock: 1-3                          [1, 256, 4, 4]            --
├─VggBlock: 1-4                          [1, 512, 2, 2]            --
├─VggBlock: 1-5                          [1, 512, 1, 1]            --
├─LinerRelu: 1-6                         [1, 256]                  --
│    └─Linear: 2-1                       [1, 256]                  131,328
│    └─ReLU: 2-2                         [1, 256]                  --
│    └─Dropout: 2-3                      [1, 256]                  --
├─LinerRelu: 1-7                         [1, 128]                  --
│    └─Linear: 2-4                       [1, 128]                  32,896
│    └─ReLU: 2-5                         [1, 128]                  --
│    └